In [1]:
import json 
import pdfplumber
import time
import jwt
import requests
import numpy as np

In [2]:
# 实际KEY，过期时间
def generate_token(apikey: str, exp_seconds: int):#
    try:
        id, secret = apikey.split(".")# 通过.分割字符串 生成id和secret 用于加密 保证数据的安全性
    except Exception as e:
        raise Exception("invalid apikey", e)

    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,  # 过期时间 当前时间+过期时间 1000是为了转换成毫秒
        "timestamp": int(round(time.time() * 1000)),  # 时间戳 当前时间 1000是为了转换成毫秒
    }
    return jwt.encode(
        payload,  # 加密的内容
        secret,  # 加密的密钥
        algorithm="HS256",  # 加密的算法
        headers={"alg": "HS256", "sign_type": "SIGN"},  # 加密的头部  alg:加密算法  sign_type:签名类型  typ:token类型
    )

In [3]:
def glm_predict( content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': generate_token("885c8c10b3f5267ab2a7dd1c7f9eb75a.qUEzlKy72Y79RMdv", 1000)
    }
    
    data = {
        "model": "glm-3-turbo",
        "messages": [  #模拟多人对话 
            {"role": "user", "content": content}
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    print("LLM Response:")
    print(response)
    return response.json()

In [5]:
questions = json.load(open('./大数据导论知识问答/submit.json', 'r', encoding='utf-8'))
pdf = pdfplumber.open('./大数据导论知识问答/大数据导论.pdf')
pdf_content = {}
for idx in range(len(pdf.pages)):
    pdf_content["page_" + str(idx + 1)] = pdf.pages[idx].extract_text()#.extract_text()提取文本

In [6]:
reference_list = [x['reference'].split('_')[1] for x in questions]
reference_list
# pdf_content['page_'+reference_list[0]].replace('\n','\t')

for idx in reference_list:
    print("idx:",idx)
    print(type(idx))
    print(pdf_content['page_' + idx].replace('\n', '\t'))
    print('-------------------'*10)
# for i in questions:
#     print(i["answer"])

idx: 24
<class 'str'>
l 第 1 章大数据概述 仁二＝＝二	运用大数据促进保障和改善民生、切实保障国家数据安全等五项工作部署，为我国发展大数据开	启了新的篇章。 2018 年 4 月 22 日至 24 日，首届“数字中国”建设峰会在福建省福州市举行（见	图 1-6 )，围绕“以信息化驱动现代化，加快建设数字中国“主题，来自各省、市、区网信部门负	责人、行业组织负责人、产业界代表、专家学者以及智库代表等约 800 人出席峰会，就建设网络	强国、数字中国、智慧社会等热点议题进行交流分享。	图 1-6 首届“数字中国”建设峰会	1.5	大数据的概念	随着大数据时代的到来，“大数据”已经成为互联网信息技术行业的流行词汇。 关于“什么是	大数据“这个问题，大家比较认可的是关于大数据的 “4V" 说法。 大数据的 4 个 “V"，或者说是	大数据的 4 个特点，包含 4个层面：数据量大 ( Volume)、数据类型繁多 (Variety)、处理速度快	(Velocity) 和价值密度低 (Value)。	1.5.1 数据量大	从数据扯的角度而言，大数据泛指无法在可容忍的时间内用传统信息技术和软 、硬件主具对	其进行获取、管理和处理的巨量数据集合，需要可伸缩的计算体系结构以支持其存储、处理和分	析。 按照这个标准来衡量，很显然，目前的很多应用场景中所涉及的数据批已经具备了大数据的	特征。 比如，微博、微信、抖音等应用平台的每天由网民发布的海最信息，属于大数据，再比如，	遍布我们飞作和生活的各个角落的各种传感器和摄像头，每时每刻都在自动产生大扯数据，这也	屈丁大数据。	13
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
idx: 16
<class 'str'>
l 第 1 章大数据概述 1	在数据爆炸的今天，人类一方面对知识充满渴求，另一方面为数据的复杂特征所困惑。 数据	爆炸对科学研究提出了更高的要求，人类需要设计出更加灵活高

In [8]:
for idx in range(len(questions)):

    prompt = '''你是一个大数据教师，帮我结合给定的资料和选项，请先判断题目的类型，再以教师的身份回答以：题目类型： 答案： 解析： 的形式回答问题。
    资料：{0}

    问题：{1}
    
    选项：{2}
        '''.format(
        pdf_content['page_'+reference_list[idx]].replace('\n','\t'),
        questions[idx]["question"],
        questions[idx]["choice"]
    )
    # print("idx:",idx)
    # print("prompt:",prompt)
    answer = glm_predict(prompt)['choices'][0]['message']['content']
    questions[idx]['answers'] = answer
    #追加写入answer.json文件
    print("idx:",idx)
with open('./大数据导论知识问答/answers_llms_prompt_optimize2.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)


LLM Response:
<Response [200]>
idx: 0
LLM Response:
<Response [200]>
idx: 1
LLM Response:
<Response [200]>
idx: 2
LLM Response:
<Response [200]>
idx: 3
LLM Response:
<Response [200]>
idx: 4
LLM Response:
<Response [200]>
idx: 5
LLM Response:
<Response [200]>
idx: 6
LLM Response:
<Response [200]>
idx: 7
LLM Response:
<Response [200]>
idx: 8
LLM Response:
<Response [200]>
idx: 9
LLM Response:
<Response [200]>
idx: 10
LLM Response:
<Response [200]>
idx: 11
LLM Response:
<Response [200]>
idx: 12
LLM Response:
<Response [200]>
idx: 13
LLM Response:
<Response [200]>
idx: 14
LLM Response:
<Response [200]>
idx: 15
LLM Response:
<Response [200]>
idx: 16
LLM Response:
<Response [200]>
idx: 17
LLM Response:
<Response [200]>
idx: 18
LLM Response:
<Response [200]>
idx: 19
LLM Response:
<Response [200]>
idx: 20
LLM Response:
<Response [200]>
idx: 21
LLM Response:
<Response [200]>
idx: 22
LLM Response:
<Response [200]>
idx: 23
LLM Response:
<Response [200]>
idx: 24
LLM Response:
<Response [200]>
idx:

In [12]:

for idx in range(len(questions)):

    prompt = '''你是一个大数据教师，帮我结合给定的资料和选项，请先判断题目的类型，再以教师的身份回答以"题目类型"： ,"答案"：注：只需要答案，不要选项内容！ ,"解析"：的json形式回答问题。
    资料：{0}

    问题：{1}
    
    选项：{2}
        '''.format(
        pdf_content['page_'+reference_list[idx]].replace('\n','\t'),
        questions[idx]["question"],
        questions[idx]["choice"]
    )
    # print("idx:",idx)
    # print("prompt:",prompt)
    answer = glm_predict(prompt)['choices'][0]['message']['content']
    questions[idx]['answers'] = answer
    #追加写入answer.json文件
    print("idx:",idx)
with open('./大数据导论知识问答/answers_llms_prompt_optimize3.json', 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)


LLM Response:
<Response [200]>
idx: 0
LLM Response:
<Response [200]>
idx: 1
LLM Response:
<Response [200]>
idx: 2
LLM Response:
<Response [200]>
idx: 3
LLM Response:
<Response [200]>
idx: 4
LLM Response:
<Response [200]>
idx: 5
LLM Response:
<Response [200]>
idx: 6
LLM Response:
<Response [200]>
idx: 7
LLM Response:
<Response [200]>
idx: 8
LLM Response:
<Response [200]>
idx: 9
LLM Response:
<Response [200]>
idx: 10
LLM Response:
<Response [200]>
idx: 11
LLM Response:
<Response [200]>
idx: 12
LLM Response:
<Response [200]>
idx: 13
LLM Response:
<Response [200]>
idx: 14
LLM Response:
<Response [200]>
idx: 15
LLM Response:
<Response [200]>
idx: 16
LLM Response:
<Response [200]>
idx: 17
LLM Response:
<Response [200]>
idx: 18
LLM Response:
<Response [200]>
idx: 19
LLM Response:
<Response [200]>
idx: 20
LLM Response:
<Response [200]>
idx: 21
LLM Response:
<Response [200]>
idx: 22
LLM Response:
<Response [200]>
idx: 23
LLM Response:
<Response [200]>
idx: 24
LLM Response:
<Response [200]>
idx: